In [ ]:
from pypokerengine.api.emulator import Emulator
from pypokerengine.players import BasePokerPlayer


# First, let's create a simple player model that always calls
class SimpleCallPlayer(BasePokerPlayer):
    def declare_action(self, valid_actions, hole_card, round_state):
        # Always call
        call_action_info = valid_actions[1]  # call is usually the second option
        return call_action_info['action'], call_action_info['amount']

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

# Set up the emulator
emulator = Emulator()
emulator.set_game_rule(player_num=2, max_round=1, small_blind_amount=5, ante_amount=0)

# Set up player information
players_info = {
    "p1": {"name": "player1", "stack": 100},
    "p2": {"name": "player2", "stack": 100},
}

# Register players
player1 = SimpleCallPlayer()
player2 = SimpleCallPlayer()
emulator.register_player("p1", player1)
emulator.register_player("p2", player2)

# Generate initial game state
initial_state = emulator.generate_initial_game_state(players_info)

# Start the round
game_state, events = emulator.start_new_round(initial_state)

# Let's see what happens in one complete round
round_state, events = emulator.run_until_round_finish(game_state)

# Print the results
print("Round finished!")
print("Final stacks:")
for player in round_state['table'].seats.players:
    print(f"{player.uuid}: {player.stack}")

Round finished!
Final stacks:
p1: 110
p2: 90


In [ ]:
from pypokerengine.api.emulator import Emulator
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.game_state_utils import restore_game_state, attach_hole_card, attach_hole_card_from_deck
from pypokerengine.utils.card_utils import gen_cards

class SimpleEmulatorPlayer(BasePokerPlayer):
    def __init__(self):
        self.emulator = Emulator()
        self.emulator.set_game_rule(player_num=2, max_round=10, small_blind_amount=5, ante_amount=0)
        self.uuid = None
        
    def declare_action(self, valid_actions, hole_card, round_state):
        # Convert string hole cards to Card objects
        hole_card = gen_cards(hole_card)
        
        # Setup emulator with same players
        my_model = OneActionModel()  # We'll define this class below
        other_model = OneActionModel()
        
        # Register players with their correct UUIDs from round_state
        for player_info in round_state['seats']:
            if player_info['uuid'] == self.uuid:
                self.emulator.register_player(player_info['uuid'], my_model)
            else:
                self.emulator.register_player(player_info['uuid'], other_model)
        
        action_results = []
        
        for action in valid_actions:
            if action['action'] in ['fold', 'call', 'raise']:
                # Create a copy of the current game state
                game_state = restore_game_state(round_state)
                
                # Attach our known hole cards
                game_state = attach_hole_card(game_state, self.uuid, hole_card)
                
                # Attach random hole cards to opponent
                for player in game_state['table'].seats.players:
                    if player.uuid != self.uuid:
                        game_state = attach_hole_card_from_deck(game_state, player.uuid)
                
                amount = action['amount']
                if action['action'] == 'raise':
                    amount = action['amount']['max']
                
                try:
                    # Set the action we want to simulate
                    my_model.set_action(action['action'], amount)
                    other_model.set_action('call', 0)  # Simple opponent model that always calls
                    
                    # Run simulation
                    game_state, events = self.emulator.run_until_round_finish(game_state)
                    
                    # Get our final stack
                    my_stack = [p.stack for p in game_state['table'].seats.players if p.uuid == self.uuid][0]
                    
                    action_results.append({
                        'action': action['action'],
                        'amount': amount,
                        'expected_stack': my_stack
                    })
                    
                except Exception as e:
                    print(f"Simulation failed for action {action['action']}: {str(e)}")
                    action_results.append({
                        'action': action['action'],
                        'amount': amount,
                        'expected_stack': 0
                    })
        
        if action_results:
            best_action = max(action_results, key=lambda x: x['expected_stack'])
            print(f"[debug_info] Simulated results: {action_results}")
            print(f"[debug_info] Choosing action: {best_action}")
            return best_action['action'], best_action['amount']
            
        # Fallback to calling if something goes wrong
        call_action = [a for a in valid_actions if a['action'] == 'call'][0]
        return call_action['action'], call_action['amount']

    def receive_game_start_message(self, game_info):
        for seat in game_info['seats']:
            if seat['name'] in ['p1', 'p2']:
                self.uuid = seat['uuid']
                break

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass
    
    def receive_street_start_message(self, street, round_state):
        pass
    
    def receive_game_update_message(self, action, round_state):
        pass
    
    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

class OneActionModel(BasePokerPlayer):
    def __init__(self):
        self.action = None
        self.amount = None
    
    def set_action(self, action, amount):
        self.action = action
        self.amount = amount
    
    def declare_action(self, valid_actions, hole_card, round_state):
        return self.action, self.amount
    
    def receive_game_start_message(self, game_info):
        pass
    
    def receive_round_start_message(self, round_count, hole_card, seats):
        pass
    
    def receive_street_start_message(self, street, round_state):
        pass
    
    def receive_game_update_message(self, action, round_state):
        pass
    
    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

if __name__ == '__main__':
    from pypokerengine.api.game import setup_config, start_poker
    
    config = setup_config(max_round=10, initial_stack=100, small_blind_amount=5)
    config.register_player(name="p1", algorithm=SimpleEmulatorPlayer())
    config.register_player(name="p2", algorithm=SimpleEmulatorPlayer())
    game_result = start_poker(config, verbose=1)

In [ ]:
from pypokerengine.api.emulator import Emulator
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.game_state_utils import restore_game_state, attach_hole_card, attach_hole_card_from_deck
from pypokerengine.utils.card_utils import gen_cards

class BetterEmulatorPlayer(BasePokerPlayer):
    def __init__(self):
        self.emulator = Emulator()
        self.emulator.set_game_rule(player_num=2, max_round=10, small_blind_amount=5, ante_amount=0)
        self.uuid = None
        self.name = None
        
    def declare_action(self, valid_actions, hole_card, round_state):
        # Print hole cards for debugging
        print(f"[debug_info] --> hole_card of emulator player is {hole_card}")
        
        # Convert hole cards to Card objects
        hole_card = gen_cards(hole_card)
        
        # Define actions to simulate
        actions_to_simulate = [
            ('fold', 0),
            ('call', valid_actions[1]['amount']),  # Call amount
            #('raise', valid_actions[2]['amount']['min']),  # Min raise
            ('raise', valid_actions[2]['amount']['max'])   # Max raise
        ]
        
        # Store results for each action
        action_results = {}
        
        # Run multiple simulations for each action
        NB_SIMULATION = 100
        for action, amount in actions_to_simulate:
            simulation_results = []
            
            for _ in range(NB_SIMULATION):
                # Setup simulation state
                game_state = restore_game_state(round_state)
                game_state = attach_hole_card(game_state, self.uuid, hole_card)
                
                # Attach random cards to opponents
                for player in game_state['table'].seats.players:
                    if player.uuid != self.uuid:
                        game_state = attach_hole_card_from_deck(game_state, player.uuid)
                
                # Setup player models for simulation
                my_model = OneActionModel()
                opponent_model = FishPlayer()  # A simple calling model
                
                # Register players
                for player_info in round_state['seats']:
                    if player_info['uuid'] == self.uuid:
                        self.emulator.register_player(player_info['uuid'], my_model)
                    else:
                        self.emulator.register_player(player_info['uuid'], opponent_model)
                
                try:
                    # Set our action for this simulation
                    my_model.set_action(action, amount)
                    
                    # Run simulation to end of round
                    game_state, events = self.emulator.run_until_round_finish(game_state)
                    
                    # Get our final stack
                    my_stack = [p.stack for p in game_state['table'].seats.players if p.uuid == self.uuid][0]
                    simulation_results.append(my_stack)
                    
                except Exception as e:
                    print(f"Simulation failed: {str(e)}")
                    continue
            
            # Calculate average stack for this action
            if simulation_results:
                avg_stack = sum(simulation_results) / len(simulation_results)
                action_results[action] = {
                    'action': action,
                    'amount': amount,
                    'average_stack': avg_stack
                }
                print(f"[debug_info] --> average stack after simulation when declares {action.upper()} : {avg_stack:.2f}")
        
        # Choose best action
        if action_results:
            best_action = max(action_results.values(), key=lambda x: x['average_stack'])
            return best_action['action'], best_action['amount']
        
        # Fallback to calling if something goes wrong
        return valid_actions[1]['action'], valid_actions[1]['amount']

    def receive_game_start_message(self, game_info):
        for seat in game_info['seats']:
            if seat['name'] in ['p1', 'p2', 'emulator_player']:
                self.uuid = seat['uuid']
                self.name = seat['name']
                break
    
    def receive_round_start_message(self, round_count, hole_card, seats):
        pass
    
    def receive_street_start_message(self, street, round_state):
        pass
    
    def receive_game_update_message(self, action, round_state):
        pass
    
    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

class OneActionModel(BasePokerPlayer):
    def __init__(self):
        self.action = None
        self.amount = None
    
    def set_action(self, action, amount):
        self.action = action
        self.amount = amount
    
    def declare_action(self, valid_actions, hole_card, round_state):
        return self.action, self.amount
    
    def receive_game_start_message(self, game_info):
        pass
    
    def receive_round_start_message(self, round_count, hole_card, seats):
        pass
    
    def receive_street_start_message(self, street, round_state):
        pass
    
    def receive_game_update_message(self, action, round_state):
        pass
    
    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

class FishPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass



if __name__ == '__main__':
    from pypokerengine.api.game import setup_config, start_poker
    
    config = setup_config(max_round=10, initial_stack=100, small_blind_amount=5)
    config.register_player(name="emulator_player", algorithm=BetterEmulatorPlayer())
    config.register_player(name="fish_player", algorithm=FishPlayer())
    game_result = start_poker(config, verbose=1)

In [9]:
from pypokerengine.api.emulator import Emulator
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.game_state_utils import restore_game_state, attach_hole_card, attach_hole_card_from_deck
from pypokerengine.utils.card_utils import gen_cards

class BetterEmulatorPlayer(BasePokerPlayer):
    def __init__(self):
        self.emulator = Emulator()
        self.emulator.set_game_rule(player_num=2, max_round=10, small_blind_amount=5, ante_amount=0)
        self.uuid = None
        self.name = None
        
    def declare_action(self, valid_actions, hole_card, round_state):
        # Print hole cards for debugging
        print(f"[debug_info] --> hole_card of emulator player is {hole_card}")
        
        # Convert hole cards to Card objects
        hole_card = gen_cards(hole_card)
        
        # Define actions to simulate
        actions_to_simulate = [
            ('fold', 0),
            ('call', valid_actions[1]['amount']),  # Call amount
        ]
        
        # Add raise action only if player has enough stack
        min_raise = valid_actions[2]['amount']['min']
        max_raise = valid_actions[2]['amount']['max']
        current_stack = next(p['stack'] for p in round_state['seats'] if p['uuid'] == self.uuid)
        
        if current_stack >= min_raise:
            actions_to_simulate.append(('raise', min(min_raise, current_stack)))  # Use min raise or available stack
        
        # Store results for each action
        action_results = {}
        
        # Run multiple simulations for each action
        NB_SIMULATION = 100
        for action, amount in actions_to_simulate:
            simulation_results = []
            
            for _ in range(NB_SIMULATION):
                # Setup simulation state
                game_state = restore_game_state(round_state)
                game_state = attach_hole_card(game_state, self.uuid, hole_card)
                
                # Attach random cards to opponents
                for player in game_state['table'].seats.players:
                    if player.uuid != self.uuid:
                        game_state = attach_hole_card_from_deck(game_state, player.uuid)
                
                # Setup player models for simulation
                my_model = OneActionModel()
                opponent_model = FishPlayer()  # A simple calling model
                
                # Register players
                for player_info in round_state['seats']:
                    if player_info['uuid'] == self.uuid:
                        self.emulator.register_player(player_info['uuid'], my_model)
                    else:
                        self.emulator.register_player(player_info['uuid'], opponent_model)
                
                try:
                    # Set our action for this simulation
                    my_model.set_action(action, amount)
                    
                    # Run simulation to end of round
                    game_state, events = self.emulator.run_until_round_finish(game_state)
                    
                    # Get our final stack
                    my_stack = [p.stack for p in game_state['table'].seats.players if p.uuid == self.uuid][0]
                    simulation_results.append(my_stack)
                    
                except Exception as e:
                    print(f"Simulation failed: {str(e)}")
                    continue
            
            # Calculate average stack for this action
            if simulation_results:
                avg_stack = sum(simulation_results) / len(simulation_results)
                action_results[action] = {
                    'action': action,
                    'amount': amount,
                    'average_stack': avg_stack
                }
                print(f"[debug_info] --> average stack after simulation when declares {action.upper()} : {avg_stack:.2f}")
        
        # Choose best action
        if action_results:
            best_action = max(action_results.values(), key=lambda x: x['average_stack'])
            return best_action['action'], best_action['amount']
        
        # Fallback to calling if something goes wrong
        return valid_actions[1]['action'], valid_actions[1]['amount']


    def receive_game_start_message(self, game_info):
        for seat in game_info['seats']:
            if seat['name'] in ['p1', 'p2', 'emulator_player']:
                self.uuid = seat['uuid']
                self.name = seat['name']
                break
    
    def receive_round_start_message(self, round_count, hole_card, seats):
        pass
    
    def receive_street_start_message(self, street, round_state):
        pass
    
    def receive_game_update_message(self, action, round_state):
        pass
    
    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

class OneActionModel(BasePokerPlayer):
    def __init__(self):
        self.action = None
        self.amount = None
    
    def set_action(self, action, amount):
        self.action = action
        self.amount = amount
    
    def declare_action(self, valid_actions, hole_card, round_state):
        return self.action, self.amount
    
    def receive_game_start_message(self, game_info):
        pass
    
    def receive_round_start_message(self, round_count, hole_card, seats):
        pass
    
    def receive_street_start_message(self, street, round_state):
        pass
    
    def receive_game_update_message(self, action, round_state):
        pass
    
    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

class FishPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass



if __name__ == '__main__':
    from pypokerengine.api.game import setup_config, start_poker
    
    config = setup_config(max_round=10, initial_stack=100, small_blind_amount=5)
    config.register_player(name="emulator_player", algorithm=BetterEmulatorPlayer())
    config.register_player(name="fish_player", algorithm=FishPlayer())
    game_result = start_poker(config, verbose=1)

Started the round 1
Street "preflop" started. (community card = [])
"fish_player" declared "call:10"
[debug_info] --> hole_card of emulator player is ['H5', 'ST']
[debug_info] --> average stack after simulation when declares FOLD : 90.00
[debug_info] --> average stack after simulation when declares CALL : 90.00
[debug_info] --> average stack after simulation when declares RAISE : 40.00
"emulator_player" declared "fold:0"
"['fish_player']" won the round 1 (stack = {'emulator_player': 90, 'fish_player': 110})
Started the round 2
Street "preflop" started. (community card = [])
[debug_info] --> hole_card of emulator player is ['S7', 'C7']
[debug_info] --> average stack after simulation when declares FOLD : 85.00
[debug_info] --> average stack after simulation when declares CALL : 80.00
[debug_info] --> average stack after simulation when declares RAISE : 30.00
"emulator_player" declared "fold:0"
"['fish_player']" won the round 2 (stack = {'emulator_player': 85, 'fish_player': 115})
Started